In [8]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.graph_objs as go 

import warnings 

warnings.filterwarnings("ignore")
plt.style.use('ggplot')

df = pd.read_csv("datasets/usd_brl_historical.csv")

df.head()

,Date,Price,Open,High,Low,Change %
0,"May 12, 2022",5.1339,5.1380,5.2102,5.1050,-0.08%
1,"May 11, 2022",5.1380,5.0992,5.1702,5.0930,0.13%
2,"May 10, 2022",5.1313,5.1622,5.1666,5.1090,-0.59%
3,"May 09, 2022",5.1617,5.0784,5.1625,5.0784,1.66%
4,"May 06, 2022",5.0774,5.0293,5.1150,5.0070,0.98%


### Feature Engineering 

In [9]:
#Rename columns 
df.columns = ['date', 'price', 'open', 'high', 'low', 'change']

#Date transform  
def convert_month(date):
    months = {
        'Jan': '01',
        'Feb': '02',
        'Mar': '03',
        'Apr': '04',
        'May': '05',
        'Jun': '06',
        'Jul': '07',
        'Aug': '08',
        'Sep': '09',
        'Oct': '10',
        'Nov': '11',
        'Dec': '12',
    }
    date[0] = months[date[0]]
    date = '-'.join(date)
    
    return date.replace(',', '') 

df.date = df.date.apply(lambda date: convert_month(date.split(' ')))

#convert to datetime 
df.date = pd.to_datetime(df.date, format='%m-%d-%Y')

df.index = df.pop('date')
df = df.iloc[::-1]

df.head()

,price,open,high,low,change
date,,,,,
2006-03-13,2.1339,2.1375,2.1445,2.1275,-0.16%
2006-03-14,2.1208,2.1315,2.1495,2.1155,-0.61%
2006-03-15,2.1157,2.1240,2.1291,2.1135,-0.24%
2006-03-16,2.1140,2.1157,2.1270,2.0970,-0.08%
2006-03-17,2.1240,2.1080,2.1286,2.1070,0.47%


### Explore data 

In [10]:
df.describe()

,price,open,high,low
count,4219.000000,4219.000000,4219.000000,4219.000000
mean,2.934794,2.933955,2.956624,2.913164
std,1.220202,1.219746,1.230951,1.208901
min,1.538300,1.538600,1.542000,1.528000
25%,1.938300,1.938750,1.953100,1.919950
50%,2.344200,2.345900,2.365900,2.324700
75%,3.768100,3.766200,3.793450,3.737950
max,5.887000,5.925100,5.972300,5.818800


In [11]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df.price, mode='lines', name='Price'))

fig.update_layout(title='Dollar exchange rate in Brazil', xaxis_title='Date', yaxis_title='Price')

In [18]:
ma = df.price.rolling(365).mean()

fig = go.Figure()

fig.add_trace(go.Scatter(x=ma.index, y=ma.values, mode='lines', name='Price'))
fig.update_layout(title='Moving Average', xaxis_title='Date', yaxis_title='Price')

In [23]:
diff = df.price.diff()

fig = go.Figure()

fig.add_trace(go.Scatter(x=diff.index, y=diff.values, mode='lines', name='Price'))
fig.update_layout(title='Difference', xaxis_title='Date', yaxis_title='Diff')

In [64]:
mean_month = df.price.diff().groupby([df.index.year, df.index.month]).mean()
mean_month.index = [f'{mean_month.index[i][0]}-{mean_month.index[i][1]}' for i in range(len(mean_month))]

fig = go.Figure()

fig.add_trace(go.Scatter(x=mean_month.index, y=mean_month.values, mode='lines', name='Price'))
fig.update_layout(title='Difference using monthly average', xaxis_title='Date', yaxis_title='Difference')